## Importing

In [1]:
import pandas as pd
import scipy as sp
import numpy as np

import emcee
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from cycler import cycler
import corner
from typing import Any, Dict, List, Optional, Tuple, Union

import lymph

## Creating or Loading Model

In [2]:
filename = "../data/extended_system.hdf5"

graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV', 'V', 'VII'],
    ('lnl', 'I'): [],
    ('lnl', 'II'): ['I', 'III', 'V', 'VII'],
    ('lnl', 'III'): ['IV'],
    ('lnl', 'IV'): [],
    ('lnl', 'V'): [],
    ('lnl', 'VII'): [],
}
extended_systm = lymph.Unilateral(graph=graph)


print(extended_systm)

Unilateral lymphatic system with 1 tumor(s) and 6 LNL(s).
primary-0.0%->VII primary-0.0%->II primary-0.0%->IV primary-0.0%->I primary-0.0%->III primary-0.0%->V II-0.0%->V II-0.0%->III II-0.0%->VII II-0.0%->I III-0.0%->IV


## Modalities

In [3]:
spsn = {"PET": [0.86, 0.79],
        "MRI": [0.63, 0.81],
        "diagnostic_consensus": [0.63, 0.81],
        "pathology": [1., 1.]}
#                         ^   ^
#                specificty   sensitivity
extended_systm.modalities = spsn

## Data

In [4]:

data = pd.read_csv("../data/latest.csv", header=[0,1,2] )
t_stage = data.iloc[:,18]
data = data.iloc[:,21:168]
ipsi_data = data.xs("ipsi",level=1,axis=1)
t_stage.loc[t_stage <= 2, ] = "early"
t_stage.loc[t_stage!="early", ] = "late"


ipsi_data["info","t_stage"]= t_stage
ipsi_data = ipsi_data.drop(['Ia', 'Ib',"IIa","IIb"], level=1, axis=1)
ipsi_data = ipsi_data.drop(['CT', 'FNA',"pCT"], level=0, axis=1)

ipsi_data.to_csv("../data/ipsi_data.csv", index=False)

extended_systm.patient_data = ipsi_data


C:\Users\Luca\AppData\Local\Temp\ipykernel_20056\1607458719.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipsi_data["info","t_stage"]= t_stage


## Storage of model

In [5]:
extended_systm.to_hdf(
    filename=filename, 
    name="extended/model"
)

C:\Users\Luca\anaconda3\envs\lymph2\lib\site-packages\lymph\utils.py:253: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->MultiIndex([(                 'MRI',       'I'),
            (                 'MRI',      'II'),
            (                 'MRI',     'III'),
            (                 'MRI',      'IV'),
            (                 'MRI',       'V'),
            (                 'MRI',     'VII'),
            (                 'PET',       'I'),
            (                 'PET',      'II'),
            (                 'PET',     'III'),
            (                 'PET',      'IV'),
            (                 'PET',       'V'),
            (                 'PET',     'VII'),
            ('diagnostic_consensus',       'I'),
            ('diagnostic_consensus',      'II'),
            ('diagnostic_consensus',     'III'),
            ('di

## Likelihood

In [6]:
#settings for the binom distributions
early_p=0.3
max_t=10
t = np.arange(max_t + 1)

def llh(theta):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    print("run")
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    
    time_dists={
        "early": sp.stats.binom.pmf(t, max_t, early_p),
        "late" : sp.stats.binom.pmf(t, max_t, late_p),
    }
     
    return extended_systm.marginal_log_likelihood(spread_probs, t_stages=["early", "late"], time_dists=time_dists)

## Sampling

In [7]:
# Settings for the sampler
ndim = len(extended_systm.spread_probs) + 1
nwalkers = 20 * ndim
nstep = 10000
burnin = 5000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]


#prepare the backend
backend = emcee.backends.HDFBackend(
    filename=filename,
    name="extended/samples"
)
backend.reset(nwalkers, ndim)


# starting point
initial_spread_probs = np.random.uniform(low=0., high=1., size=(nwalkers,ndim))


if __name__ == "__main__":
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, 
        llh, 
        moves=moves,
        backend=backend
    )
    sampler.run_mcmc(initial_spread_probs, nstep, progress=True)
    samples_HMM = sampler.get_chain(flat=True, discard=burnin)
    print(samples_HMM)      




run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


 20%|██        | 2/10 [00:00<00:00, 19.80it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


 60%|██████    | 6/10 [00:00<00:00, 16.37it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


...

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


100%|██████████| 10/10 [00:00<00:00, 15.89it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


100%|██████████| 10/10 [00:00<00:00, 16.03it/s]

[[0.7085506  0.24891566 0.5312608  ... 0.20346184 0.78069308 0.6275805 ]
 [0.7450237  0.35668552 0.62526514 ... 0.85880199 0.04437147 0.32984468]
 [0.04078955 0.41070053 0.90435865 ... 0.0389364  0.6994415  0.57054242]
 ...
 [0.8309393  0.50403191 0.23338386 ... 0.74398234 0.53009747 0.22302256]
 [0.27429036 0.32476246 0.28723437 ... 0.80912811 0.69440279 0.78116214]
 [0.02001092 0.42238799 0.08707033 ... 0.25908282 0.3030518  0.21193677]]
